In [ ]:
import os
import shutil
import random
from PIL import Images



In [ ]:
C:\Users\Maxi\Documents\Forschsem\Riseholme-2021\Data\Normal\Ripe

In [ ]:
def create_split(image_folder, train_image_folder, val_image_folder, train_label_folder, val_label_folder):
    # Prüfen, ob die Zielordner existieren oder leer sind
    if not os.path.exists(train_image_folder) or not os.listdir(train_image_folder):
        os.makedirs(train_image_folder, exist_ok=True)
    else:
        print(f"Trainingsordner '{train_image_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    if not os.path.exists(val_image_folder) or not os.listdir(val_image_folder):
        os.makedirs(val_image_folder, exist_ok=True)
    else:
        print(f"Testordner '{val_image_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    # Auch für die Label-Ordner überprüfen
    if not os.path.exists(train_label_folder) or not os.listdir(train_label_folder):
        os.makedirs(train_label_folder, exist_ok=True)
    else:
        print(f"Trainingslabel-Ordner '{train_label_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    if not os.path.exists(val_label_folder) or not os.listdir(val_label_folder):
        os.makedirs(val_label_folder, exist_ok=True)
    else:
        print(f"Testlabel-Ordner '{val_label_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    # Hole die Liste der Bilddateien im Quellordner
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

    # Vergewissere dich, dass es tatsächlich Bilder gibt
    if len(image_files) == 0:
        print("Es gibt keine Bilddateien im Quellordner. Das Skript wird abgebrochen.")
        return

    # Zufällige Auswahl von 70% für Training und 30% für Test
    random.shuffle(image_files)

    # Berechne die Aufteilung (70% Training, 30% Test)
    split_index = int(0.7 * len(image_files))
    train_images = image_files[:split_index]
    val_images = image_files[split_index:]

    # Kopiere die Bilder und Label-Dateien in die entsprechenden Ordner
    for img in train_images:
        # Kopiere das Bild
        shutil.copy(os.path.join(image_folder, img), os.path.join(train_image_folder, img))
        # Kopiere die zugehörige Label-Datei
        label_file = img.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(image_folder.replace("images", "labels"), label_file), os.path.join(train_label_folder, label_file))

    for img in val_images:
        # Kopiere das Bild
        shutil.copy(os.path.join(image_folder, img), os.path.join(val_image_folder, img))
        # Kopiere die zugehörige Label-Datei
        label_file = img.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(image_folder.replace("images", "labels"), label_file), os.path.join(val_label_folder, label_file))

    print(f"Datensatz erfolgreich aufgeteilt: {len(train_images)} Bilder für Training, {len(val_images)} Bilder für Validierung.")

# Beispiel: Definiere deine Pfade
image_folder = "/path/to/your/images"  # Pfad zu den Bilddateien
train_image_folder = "/path/to/your/train_images"
val_image_folder = "/path/to/your/val_images"
train_label_folder = "/path/to/your/train_labels"
val_label_folder = "/path/to/your/val_labels"

# Führe die Funktion aus, um den Datensatz zu teilen
create_split(image_folder, train_image_folder, val_image_folder, train_label_folder, val_label_folder)


In [ ]:
def create_yolo_labels(image_folder, label_folder):
    """
    Erstellt YOLO-Label-Dateien für jedes Bild im angegebenen Ordner.
    - Annahme: Jede Erdbeere ist eine Klasse 0.
    - Die Bounding Boxes liegen am Rand der Bilder.

    :param image_folder: Pfad zum Ordner mit den Bildern
    :param label_folder: Pfad zum Ordner, in dem die Label-Dateien gespeichert werden
    """
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

    # Alle Bilder im angegebenen Ordner durchlaufen
    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.jpg', '.png', '.jpeg')):  # Unterstützte Bildformate
            image_path = os.path.join(image_folder, image_name)
            image = Image.open(image_path)
            width, height = image.size  # Bildbreite und -höhe

            # Label-Datei im YOLO-Format erstellen
            label_filename = os.path.splitext(image_name)[0] + '.txt'
            label_path = os.path.join(label_folder, label_filename)

            # YOLO-Format erfordert relative Koordinaten (normalisiert)
            # In diesem Fall nehmen wir an, dass Erdbeeren die gesamte Randbereich des Bildes abdecken.
            
            # Beispiel: 4 Bounding Boxen für die Ränder des Bildes
            bounding_boxes = [
                (0, 0, width / 2, height / 2),  # Oben links
                (width / 2, 0, width / 2, height / 2),  # Oben rechts
                (0, height / 2, width / 2, height / 2),  # Unten links
                (width / 2, height / 2, width / 2, height / 2)  # Unten rechts
            ]

            # Alle Bounding Boxes in YOLO-Format umwandeln und speichern
            with open(label_path, 'w') as label_file:
                for box in bounding_boxes:
                    x_center = (box[0] + box[2] / 2) / width
                    y_center = (box[1] + box[3] / 2) / height
                    box_width = box[2] / width
                    box_height = box[3] / height

                    # Klasse 0 für Erdbeeren
                    label_file.write(f"0 {x_center} {y_center} {box_width} {box_height}\n")
            
            print(f"Label für {image_name} erstellt.")

# Beispielaufruf der Funktion
create_yolo_labels('path/to/images', 'path/to/labels')
